# Ochroniarz - Wykrywanie nieodpowiednich treści i wrażliwych danych

In [1]:
##### == Import libraries ==
from openai import OpenAI, AuthenticationError
import spacy
import sys
import textwrap

# == Global variables ==
END_MES = "====Program zakonczony===="
# * flags
detected_harassment_or_threats = False
detected_sensitive_data = False


In [2]:
# == Try to open file and read it to text variable ==
try:
    text = open("text/text1.txt", "r").read().strip().strip("\n")
except FileNotFoundError:
    print(f"Nie znaleziono pliku: {sys.argv[1]}")
    sys.exit(2)
# print text if file found
print("-"*25, "TREŚĆ DOKUMENTU","-"*25 )
print(text)
print("-"*30, "END","-"*30 )

------------------------- TREŚĆ DOKUMENTU -------------------------
Jane Smith, born April 12, 1990, resides at 4321 Oak Avenue, Rivertown, TX, 75001. She is employed by Global Solutions Ltd., located at 8765 Pine Boulevard, Orlando, FL, 32801. Her Social Security number is 666-66-6666, and her credit card number is 1234-5678-9123-4567, with an expiration date of 06/28 and a CVV of 456. Jane's driver's license number is X1234568, issued in Texas. She holds a bank account number 123456789 at National Trust Bank. Hateful text: I hate you, you are SO BAAAAAD! You are useless! You are worthless!
------------------------------ END ------------------------------


## <u>Wykrywanie mowy nienawiści/gróźb/zastraszania/molestowania</u>

    Do wykrywania nieodpowiednich treści w tekście skorzystałem z API OpenAI Moderations. Jest to model specjalnie przeznaczony do tego zadania, Pozwala ocenić treści na skali od 0 do 1, co ułatwia monitorowanie i filtrację mowy nienawiści. 

    Do zadziałania programu wymagane jest wpisanie swojego klucza API do pliku `API_KEY"

`Zmiana tekstu do testowania:`

In [3]:
text = text

In [4]:
# == Initialize OpenAi library ==
# * read API key  from file
API_KEY = open("API_KEY", "r").read()
# * create OpenAi client
client = OpenAI(
    api_key=API_KEY)
# * try to get a response from moderations endpoint
try:
    response = client.moderations.create(
        input=text)
except (AuthenticationError, NameError):
    response = ""
    while True:
        print("Error: Błędny klucz API, czy chcesz kontynuować bez sprawdzania mowy nienawiści?")
        users_input = input("Wpisz Y/N: ")
        if users_input.lower() == "y":
            break
        elif users_input.lower() == "n":
            sys.exit(END_MES)

In [5]:
# == Hate speech recognition - OpenAI moderations ==
# check response if there is one
if response:
    # * load all categories
    categories = response.results[0].categories
    categories_dict = vars(categories)  # * access attributes as dict
    # * load all categories scores
    category_scores = response.results[0].category_scores
    category_scores_dict = vars(category_scores)  # * access attributes as dict
    # print if hate speech detected
    for key in categories_dict:
        if categories_dict[key] == True:

            # * Set flag to True when something detected
            detected_harassment_or_threats = True
            # * print message
            print("❗\033[31mWykryto", key, "==",  categories_dict[key], "!", end=" ")
            print(f"\033[0mOkreślona wartość {round(category_scores_dict[key], 2)} na 1.0"

                  )
        else:
            # * if False continue to the next key
            continue
    if not detected_harassment_or_threats:
        print(
            "✅\033[32m Nie znaleziono mowy nienawiści/gróźb/zastraszania/molestowania \033[0m")

else:
    print("====Kontynuacja bez sprawdzania mowy nienawiści====")




❗Wykryto harassment == True ! Określona wartość 0.5 na 1.0


## <u> Wykrywanie wrażliwych danych </u>

    Do wykrywania wrażliwych danych skorzystałem z popularnej biblioteki spacy do przetwarzania języka naturalnego. Posiada ona możliwość wyszukiwania fragmentów tekstu odnoszących się do do imion osób, nazw miejsc, firm, numerow telefonów, kwot pieniężnych, dat, i wielu innych specyficznych informacji.

`Zmiana tekstu do testowania:`

In [6]:
text = text

### Stworzenie słownika zawierającego wrażliwe treści i unikalne zamienniki (placeholdery)

In [7]:
# *initialize spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
# create dictionary for replacing words for placeholders {"Entity": "placeholder", ...}
dataTransform_dict = {}
# value index - for making values in dictionary UNIQUE
count = 0
for ent in doc.ents:
    if ent.text in dataTransform_dict:
        # *if key exists, just continue to the nest ent
        continue
    else:
        # * Set flag to True when something detected
        detected_sensitive_data = True
        # * add key/value pair dictionary with unique value count
        dataTransform_dict[ent.text] = ent.label_ + f"[{count}]"
        count += 1

if not detected_sensitive_data:
    print("✅\033[32m Nie znaleziono wrażliwych danych \033[0m")
else:
    print("❗\033[31mWykryto wrażliwe dane \033[0m")


❗Wykryto wrażliwe dane 


In [8]:
#### Słownik zamiany wrażliwych danych na anonimowe

In [9]:
# print("Słownik zmiany wrażliwych danych na anonimowe:", dataTransform_dict)
print("-"*25, "SŁOWNIK ZAMIANY","-"*25 )
for key in dataTransform_dict:
    print(f"'{key}': '{dataTransform_dict[key]}',")
print("-"*25, "END","-"*25 )

------------------------- SŁOWNIK ZAMIANY -------------------------
'Jane Smith': 'PERSON[0]',
'April 12, 1990': 'DATE[1]',
'4321': 'DATE[2]',
'Oak Avenue': 'FAC[3]',
'Rivertown': 'GPE[4]',
'TX': 'ORG[5]',
'75001': 'PERSON[6]',
'Global Solutions Ltd.': 'ORG[7]',
'8765 Pine Boulevard': 'FAC[8]',
'Orlando': 'GPE[9]',
'FL': 'ORG[10]',
'32801': 'DATE[11]',
'Social Security': 'ORG[12]',
'666': 'CARDINAL[13]',
'1234-5678-9123-4567': 'DATE[14]',
'06/28': 'CARDINAL[15]',
'CVV': 'ORG[16]',
'456': 'CARDINAL[17]',
'Jane': 'PERSON[18]',
'Texas': 'GPE[19]',
'123456789': 'DATE[20]',
'National Trust Bank': 'ORG[21]',
------------------------- END -------------------------


### Funkcja zamieniająca substringi obecne w dict.keys() na placeholdery 

In [10]:
def replace_entities_with_placeholders(text, dictionary):
    """
    returns text with sensitive data replaced with placeholders
    """
    # Get the dictionary keys
    dict_keys = dictionary.keys()

    # create 2 versions for replacing text
    text_not_highlited = text
    text_highlited = text
    # replace each key with placeholder
    for key in dict_keys:
        text_not_highlited = text_not_highlited.replace(key, f"{dictionary[key]}")
        text_highlited = text_highlited.replace(key, f"\033[32m{dictionary[key]}\033[0m")
    return text_not_highlited, text_highlited

In [11]:
changed_text, changed_text_highlited = replace_entities_with_placeholders(text, dataTransform_dict)
print("=========Oryginalny tekst===========")
print(text)
print("=========Zmieniony tekst===========")
print(changed_text_highlited + "\n" + "="*70)

=========Oryginalny tekst===========
Jane Smith, born April 12, 1990, resides at 4321 Oak Avenue, Rivertown, TX, 75001. She is employed by Global Solutions Ltd., located at 8765 Pine Boulevard, Orlando, FL, 32801. Her Social Security number is 666-66-6666, and her credit card number is 1234-5678-9123-4567, with an expiration date of 06/28 and a CVV of 456. Jane's driver's license number is X1234568, issued in Texas. She holds a bank account number 123456789 at National Trust Bank. Hateful text: I hate you, you are SO BAAAAAD! You are useless! You are worthless!
=========Zmieniony tekst===========
PERSON[0], born DATE[1], resides at DATE[2] FAC[3], GPE[4], ORG[5], PERSON[6]. She is employed by ORG[7], located at FAC[8], GPE[9], ORG[10], DATE[11]. Her ORG[12] number is CARDINAL[13]-66-CARDINAL[13]6, and her credit card number is DATE[14], with an expiration date of CARDINAL[15] and a ORG[16] of CARDINAL[17]. PERSON[18]'s driver's license number is X123CARDINAL[17]8, issued in GPE[19]. Sh

### Funkcja zamieniająca placeholdery na oryginalny tekst

In [12]:
def replace_placeholders_with_entities(text, dictionary):
    """
    returns text with placeholders replaced to oryginal text
    """
    # Reverse dictionary keys->values, values->keys using dictionary comprehension
    reverse_dict = {v: k for k, v in dictionary.items()}
     # Get the reversed_dictionary keys
    reverse_dict_keys = reverse_dict.keys()

    # strip text from color 
    text = text.strip("\033[32m").strip("\033[0m")
    # replace each placeholder in text
    for key in reverse_dict_keys:
        text = text.replace(key, f"{reverse_dict[key]}")

    return text

In [13]:
oryginal_text = replace_placeholders_with_entities(text, dataTransform_dict)
print("========Tekst z placeholderami============")
print(changed_text_highlited)
print("=========Odszyfrowany tekst===========")
print(oryginal_text + "\n" + "="*70)

========Tekst z placeholderami============
PERSON[0], born DATE[1], resides at DATE[2] FAC[3], GPE[4], ORG[5], PERSON[6]. She is employed by ORG[7], located at FAC[8], GPE[9], ORG[10], DATE[11]. Her ORG[12] number is CARDINAL[13]-66-CARDINAL[13]6, and her credit card number is DATE[14], with an expiration date of CARDINAL[15] and a ORG[16] of CARDINAL[17]. PERSON[18]'s driver's license number is X123CARDINAL[17]8, issued in GPE[19]. She holds a bank account number 123CARDINAL[17]789 at ORG[21]. Hateful text: I hate you, you are SO BAAAAAD! You are useless! You are worthless!
=========Odszyfrowany tekst===========
Jane Smith, born April 12, 1990, resides at 4321 Oak Avenue, Rivertown, TX, 75001. She is employed by Global Solutions Ltd., located at 8765 Pine Boulevard, Orlando, FL, 32801. Her Social Security number is 666-66-6666, and her credit card number is 1234-5678-9123-4567, with an expiration date of 06/28 and a CVV of 456. Jane's driver's license number is X1234568, issued in Tex

### Funkcja zapisująca nowy tekst i słownik do osobnego pliku

In [14]:
def create_file_newText_dict(newText, dictionary):
    """
    creates a file called "result.txt" with newText and dictionary
    """
    # open or create new file called result.txt
    with open("result.txt", "w") as file:
        # write text into the file
        file.write("="*25 + "Zmieniony tekst" + "="*25 + "\n") # separator
        file.write(newText + "\n")
        # convert distionary to str and write into the file
        file.write("="*25 + "Slownik zamiany" + "="*25 + "\n") # separator
        file.write(str(dictionary) + "\n")
        file.write("="*70 + "\n") # separator
        
    

In [15]:
# create_file_newText_dict(changed_text, dataTransform_dict)
create_file_newText_dict(changed_text, dataTransform_dict)